In [ ]:
%matplotlib inline
# import
import seaborn as sns
import numpy as np
import pandas as pd
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import json
import os
import pathlib
from matplotlib.colors import ListedColormap

net_bpi15 = "../../../eval-results/_server-results/net-BP-bpi15/net.eval" 
log_bpi15 = "../../../eval-results/_server-results/test35_ppp/"
nolog_bpi15 = "../../../eval-results/download-server/DF-p"
#nolog_bpi15 = "../../../eval-results/_server-results/test28_ppp/"

In [ ]:
def net1adder(row):
    #get net names
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return row['Name'].split('-')[0].replace(".pnml","")
    else:
        return 0
def net2adder(row):        
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return row['Name'].split('-')[1].split('.')[0].replace(".pnml","")
    else: 
        return 0
    
def pnml_remover(row):
    #get net names
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return row['Name'].replace(".pnml","").replace(".rdf","")
    else:
        return row['Name']

def net1NonTaus(row):
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return df_nets.at[row['net1'],'nNonSilentTransitions'] 
    else:
        return 0
def net2NonTaus(row):
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return df_nets.at[row['net2'],'nNonSilentTransitions'] 
    else:
        return 0
def sumNonTaus(row):    
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return row['net1NonTaus'] + row['net2NonTaus']
    else:
        return 0

def fetch_data(folder, net_analysis, dataset, profile):
    dir_list = next(os.walk(folder))[1]
    dfs = []
    #merge all 
    for subfolder in dir_list:
        #print(subfolder)
        evalFile = folder +"/" + subfolder +"/aggRetrospectiveResults.eval"
        confFile = folder +"/" + subfolder +"/config.log"
        if os.path.exists(evalFile):
            df = pd.read_csv(evalFile ,encoding="ISO-8859-1", skipinitialspace=True)
            with open(confFile) as json_file:
                conf = json.load(json_file)
                #add config information to dataframe 
                df['matcher'] = conf['matcher']['ilp'] + " - " + conf['matcher']['profile'] +" - " + str(conf['matcher']['word-sim']) + " - sim-weight=" + str(conf['matcher']['sim-weight']) + " - match-postprocessing=" + str(conf['matcher']['postprocessing-thresh'] ) + " - Node Limit: " +str(conf['matcher']['ilp-node-limit']) + "- Time Limit : " +str(conf['matcher']['ilp-time-limit'])
                df['matcher_wo_weight'] = conf['matcher']['ilp'] + " - " + conf['matcher']['profile'] +" - " + str(conf['matcher']['word-sim']) + " - match-postprocessing=" + str(conf['matcher']['postprocessing-thresh']) + " - Node Limit: " +str(conf['matcher']['ilp-node-limit']) + "- Time Limit : " +str(conf['matcher']['ilp-time-limit'])
                df['matcher_wo_limit'] = conf['matcher']['ilp'] + " - " + conf['matcher']['profile'] +" - " + str(conf['matcher']['word-sim']) + " - sim-weight=" + str(conf['matcher']['sim-weight']) + " - match-postprocessing=" + str(conf['matcher']['postprocessing-thresh'] )
                df['complex-matches'] = conf['matcher']['complex matches']
                df['profile'] = conf['matcher']['profile']
                df['ilp'] =  conf['matcher']['ilp']
                df['word-sim'] =  conf['matcher']['word-sim']
                df['sim-weight'] = conf['matcher']['sim-weight']
                df['matcher-postprocessing-threshold'] = conf['matcher']['postprocessing-thresh']
                df['eval-postprocessing-threshold'] = conf['evaluation']['postprocessing-thresh']
                df['ILP-time-limit'] = conf['matcher']['ilp-time-limit']
                df['ILP-node-limit'] = conf['matcher']['ilp-node-limit']
            
            #df.set_index(['Name','matcher'])
            dfs.append(df)

    df_combined = pd.concat(dfs)

    #convert time
    df_combined['OVERALL TIME'] = df_combined['OVERALL TIME'].map(lambda x: x / 1000000000.)
    df_combined['BP TIME'] = df_combined['BP TIME'].map(lambda x: x / 1000000000.)
    df_combined['LABEL-SIM TIME'] = df_combined['LABEL-SIM TIME'].map(lambda x: x / 1000000000.)
    df_combined['LP TIME'] = df_combined['LP TIME'].map(lambda x: x / 1000000000.)
    df_combined['Dataset'] = dataset
    df_combined['Profile'] = profile

    #extend with net information stored in net_analysis
    df_nets = pd.read_csv(net_analysis ,encoding="ISO-8859-1", skipinitialspace=True)
    #df_nets.set_index('Name')


    df_combined['net1'] = df_combined.apply(lambda row: net1adder(row), axis=1)
    df_combined['net2'] = df_combined.apply(lambda row: net2adder(row), axis=1)
    #df_combined['net1NonTaus'] = df_combined.apply(lambda row: net1NonTaus(row), axis=1)
    #df_combined['net2NonTaus'] = df_combined.apply(lambda row: net2NonTaus(row), axis=1)
    #df_combined['sumNonTaus'] = df_combined.apply(lambda row: sumNonTaus(row), axis=1)
    #df_combined['Name'] = df_combined.apply(lambda row: pnml_remover(row), axis=1)
    return df_combined

In [ ]:
# load 
df_bpi15Log = fetch_data(log_bpi15, net_bpi15 ,"BPI15", "Log-Log")
df_bpi15NoLog = fetch_data(nolog_bpi15, net_bpi15 ,"BPI15", "Model-Model")

# concat all three datasets
df_all = pd.concat([df_bpi15Log, df_bpi15NoLog])

df_macro_avg = df_all[df_all.Name == "Aggregated (MACRO)"]
df_micro_avg = df_all[df_all.Name == "Aggregated (MICRO)"]

#select max fscore for each profile on each dataset
df_macro_max_fscore = df_macro_avg.groupby(['Dataset','Profile']).max()["FSCORE"]
df_macro_max_fscore = df_macro_max_fscore.reset_index()

df_micro_max_fscore = df_micro_avg.groupby(['Dataset','Profile']).max()["FSCORE"]
df_micro_max_fscore = df_micro_max_fscore.reset_index()

#export dataframe to excel
with pd.ExcelWriter('log-model-postprocessing.xlsx') as writer:  
    df_macro_avg.to_excel(writer, sheet_name='Log-Model Macro')  
    df_micro_avg.to_excel(writer, sheet_name='Log-Model Micro')  

fig, (ax1, ax2) = plt.subplots(1,2)
fig.set_size_inches(18.5, 5.25)
sns.lineplot(x="eval-postprocessing-threshold", y="PRECISION", hue="Profile", data=df_macro_avg, ax = ax1, palette= sns.color_palette("Reds_d", 2));
ax1.set_title("Macro")
ax1.set_ylabel("Precision")
ax1.set_ylim([0.0,1.0])
ax1.set_xlabel("Post-processing Threshold")
sns.lineplot(x="eval-postprocessing-threshold", y="PRECISION", hue="Profile", data=df_micro_avg, ax = ax2, palette= sns.color_palette("Reds_d", 2));
ax2.set_title("Micro")
ax2.set_ylabel("Precision")
ax2.set_ylim([0.0,1.0])
ax2.set_xlabel("Post-processing Threshold")
plt.show()

fig, (ax1, ax2) = plt.subplots(1,2)
fig.set_size_inches(18.5, 5.25)
sns.lineplot(x="eval-postprocessing-threshold", y="RECALL", hue="Profile", data=df_macro_avg, ax = ax1, palette= sns.color_palette("Reds_d", 2));
ax1.set_title("Macro")
ax1.set_ylabel("Recall")
ax1.set_ylim([0.0,1.0])
ax1.set_xlabel("Post-processing Threshold")
sns.lineplot(x="eval-postprocessing-threshold", y="RECALL", hue="Profile", data=df_micro_avg, ax = ax2, palette= sns.color_palette("Reds_d", 2));
ax2.set_title("Micro")
ax2.set_ylabel("Recall")
ax2.set_ylim([0.0,1.0])
ax2.set_xlabel("Post-processing Threshold")
plt.show()

#print(df_macro_max_fscore)
fig, (ax1, ax2) = plt.subplots(1,2)
fig.set_size_inches(18.5, 5.25)
sns.lineplot(x="eval-postprocessing-threshold", y="FSCORE", hue="Profile", data=df_macro_avg, ax = ax1, palette= sns.color_palette("Reds_d", 2));
ax1.set_title("Macro")
ax1.set_ylabel("Fscore")
ax1.set_ylim([0.0,1.0])
ax1.set_xlabel("Post-processing Threshold")
sns.lineplot(x="eval-postprocessing-threshold", y="FSCORE", hue="Profile", data=df_micro_avg, ax = ax2, palette= sns.color_palette("Reds_d", 2));
ax2.set_title("Micro")
ax2.set_ylabel("Fscore")
ax2.set_ylim([0.0,1.0])
ax2.set_xlabel("Post-processing Threshold")
plt.show()

In [ ]:
# load 
df_bpi15Log = fetch_data(log_bpi15, net_bpi15 ,"BPI15", "Log-Log")
df_bpi15NoLog = fetch_data(nolog_bpi15, net_bpi15 ,"BPI15", "Model-Model")

# concat all three datasets
df_all = pd.concat([df_bpi15Log, df_bpi15NoLog])

#filter out aggregations
df_all = df_all[(df_all["Name"] != "Aggregated (MACRO)") & (df_all["Name"] != "Aggregated (MICRO)")]

#filter on pp threshold
df_all = df_all[df_all["eval-postprocessing-threshold"] == 0.00]

#sorting according to log
ordering = df_all[df_all["Profile"] == "Log-Log"].sort_values("FSCORE")["Name"]

fig, (ax1) = plt.subplots(1,1)
fig.set_size_inches(9.25, 5.25)
sns.pointplot(x="Name", y="FSCORE", hue="Profile", data=df_all, ax = ax1, palette= sns.color_palette("Reds_d", 2), order = ordering);
ax1.set_xticks([])
ax1.set_ylabel("Fscore")
ax1.set_xlabel("Pair of models")


In [ ]:
# load 
df_bpi15Log = fetch_data(log_bpi15, net_bpi15 ,"BPI15", "log-based matcher")
df_bpi15NoLog = fetch_data(nolog_bpi15, net_bpi15 ,"BPI15", "model-based matcher")

# concat all three datasets
df_all = pd.concat([df_bpi15Log, df_bpi15NoLog])


#filter out aggregations
df_all = df_all[(df_all["Name"] != "Aggregated (MACRO)") & (df_all["Name"] != "Aggregated (MICRO)")]

#filter on pp threshold
df_all = df_all[df_all["eval-postprocessing-threshold"] == 0.00]

#export dataframe to excel
with pd.ExcelWriter('log-model.xlsx') as writer:  
    df_all.to_excel(writer, sheet_name='Log-Model')  

#sorting according to log
df1 = df_all[df_all["Profile"] == "log-based matcher"][["Name","FSCORE"]]
df2 = df_all[df_all["Profile"] == "model-based matcher"][["Name","FSCORE"]]
df2.rename(index=str, columns={'FSCORE': 'FSCORE2'}, inplace=True)
ordering = pd.merge(df1,df2, on=["Name"]).sort_values(["FSCORE","FSCORE2"])

fig, (ax1) = plt.subplots(1,1)
plt.legend(loc=4)
fig.set_size_inches(9.25, 5.25)
sns.pointplot(x="Name", y="FSCORE", hue="Profile", data=df_all[df_all["Profile"] == "log-based matcher"],scale=0.5, ax = ax1, palette= sns.color_palette("Blues_d", 1),dodge=False, order = ordering["Name"]);
sns.pointplot(x="Name", y="FSCORE", hue="Profile", data=df_all[df_all["Profile"] == "model-based matcher"],scale=0.5, ax = ax1, palette= sns.color_palette("Reds_d", 1), dodge=False, order = ordering["Name"]);

newXlabel =  list(range(10, 201,10))
plt.gca().set_xticklabels(newXlabel)

plt.xticks(np.arange(min(newXlabel), max(newXlabel)+1, 10.0))

plt.legend(loc=4, title= "Matcher")

#ax1.set_xticks([1,203])
ax1.set_ylim([-0.01,1.0])
ax1.set_ylabel("Fscore")
ax1.set_xlabel("Pair of models, sorted by (1) fscore of the log-based matcher (2) fscore of the model-based matcher")
ax1.set_title("Fscore per model pair")



In [ ]:
#Analysis where the Model based one perfomed better:
df_bpi15Log = fetch_data(log_bpi15, net_bpi15 ,"BPI15", "Log-Log")
df_bpi15NoLog = fetch_data(nolog_bpi15, net_bpi15 ,"BPI15", "Model-Model")

df_bpi15Log = df_bpi15Log[(df_bpi15Log["Name"] != "Aggregated (MACRO)") & (df_bpi15Log["Name"] != "Aggregated (MICRO)")]
df_bpi15NoLog = df_bpi15NoLog[(df_bpi15NoLog["Name"] != "Aggregated (MACRO)") & (df_bpi15NoLog["Name"] != "Aggregated (MICRO)")]

df_bpi15Log = df_bpi15Log[df_bpi15Log["eval-postprocessing-threshold"] == 0.00]
df_bpi15NoLog = df_bpi15NoLog[df_bpi15NoLog["eval-postprocessing-threshold"] == 0.00]
cnt = 0
for (rowName, rowLog) in df_bpi15Log.iterrows():
    rowModel = df_bpi15NoLog[df_bpi15NoLog["Name"] == rowLog["Name"]].iloc[0]
    if  rowModel.FSCORE > rowLog.FSCORE and rowLog.FSCORE ==0:
        print(rowModel["Name"] + str(rowLog.FSCORE) +"<"+ str(rowModel.FSCORE) + "                 "+str(rowModel.FSCORE -   rowLog.FSCORE))
        cnt+=1
print(cnt)
       
    